In [ ]:
import pandas as pd 
import numpy as np
import datetime

In [ ]:
df = pd.read_csv("../csv/SPY.csv")
df.head(1)

In [ ]:
price_df = df.loc[:, ['Date', 'Adj Close']]
price_df.head(1)

In [ ]:
## 파생변수 '년-월'을 값으로 컬럼의 이름은 'STD-YM'

# case1
# Date 컬럼의 값을 datetime형으로 변환
# dt.strftime() 년-월 추출
# price_df["Date"] = pd.to_datetime(price_df["Date"])
# price_df["STD-YM"] = price_df["Date"].dt.strftime("%Y-%m")
# price_df.head(1) 

# case2
# datetime 라이브러리 사용하여 추출
price_df["STD-YM"] = price_df["Date"].map(lambda x : \
    datetime.datetime.strptime(x, '%Y-%m-%d').strftime('%Y-%m'))
price_df.head(1)

In [ ]:
## 새로 만든 파생변수 STD-YM 값에서 유니크값을 리스트 형태로 추출
month_list = price_df["STD-YM"].unique()
month_list

In [ ]:
month_last_df = pd.DataFrame()

## 년,월별 마지막날의 데이터들을 month_last_df에 삽입
## 반복문 month_list값을 기준으로 반복
## 해당 년월을 기준으로 데이터를 필터링
## 그 값들중에 인덱스가 가장 마지막에 있는 데이터를 month_list_df 삽입
#case1
# price_df.loc[price_df[price_df["STD-YM"] == month_list[1]].index[-1]]
#case2
# price_df[price_df["STD-YM"] == month_list[1]].tail(1)

# for i in month_list:
#     month_last_df = month_last_df.append(price_df.loc[\
#         price_df[price_df["STD-YM"] == i].index[-1]])
# month_last_df.head()

for i in month_list:
    last_df = price_df[price_df["STD-YM"] == i].tail(1)
    month_last_df = pd.concat([month_last_df, last_df])
month_last_df.head()

In [ ]:
## 'Date'컬럼을 인덱스 변경
month_last_df.set_index(['Date'], inplace=True)
month_last_df.head(1)

In [ ]:
## 파생변수 2개 생성
## 'BF_1M' 전월의 종가의 값을 가지는 파생변수
## 'BF_12M' 전년도의 종가의 값을 가지는 파생변수
## Nan값은 0으로 채운다.

month_last_df["BF_1M"] = month_last_df.shift(1)["Adj Close"].fillna(0)
month_last_df["BF_12M"] = month_last_df.shift(12)["Adj Close"].fillna(0)
# month_last_df = month_last_df.fillna(0)
month_last_df.head(14)

In [ ]:
## price_df에 trade 컬럼을 추가 
price_df["trade"] = ""
price_df.set_index(['Date'], inplace=True)

In [ ]:
# 거래내역 추가 
for i in month_last_df.index:
    signal = ""
    #절대 모멘텀 계산
    momentum_index = month_last_df.loc[i, "BF_1M"] / month_last_df.loc[i, "BF_12M"] - 1
    print(momentum_index)
    # 절대 모멘텀 지표에 따라서 True / False 판단
    flag = True if ((momentum_index > 0.0) and (momentum_index != np.inf) \
        and (momentum_index != -np.inf)) else False
    # if (momentum_index > 0.0) and (momentum_index != np.inf) and (momentum_index != -np.inf):
    #     flag = True
    # else : 
    #     flag = False
    if flag:
        signal = "buy"
    print("날짜 : ", i, "모멘텀 인덱스 : ", momentum_index, "flag : ", flag, "signal : ", signal)
    price_df.loc[i, "trade"] = signal
price_df.value_counts('trade')

In [ ]:
rtn = 1.0
price_df["return"] = 1
buy = 0.0
sell = 0.0
for i in price_df.index:
    if price_df.loc[i, "trade"] == "buy" and price_df.shift(1).loc[i, "trade"] == "":
        buy = price_df.loc[i, "Adj Close"]
        print("진입일 : ", i, "진입 가격 : ", buy)
    elif price_df.loc[i, "trade"] == "" and price_df.shift(1).loc[i, "trade"] == "buy":
        sell = price_df.loc[i, "Adj Close"]
        rtn = (sell - buy) / buy + 1
        price_df.loc[i, "return"] = rtn
        print("청산일 : ", i, "진입 가격 : ", buy, "청산 가격 : ", sell,\
             "| return : ", round(rtn, 4))
    if price_df.loc[i, "trade"] == "":
        buy = 0.0
        sell = 0.0
acc_rtn = 1.0
for i in price_df.index:
    rtn = price_df.loc[i, "return"]
    acc_rtn = acc_rtn * rtn
    price_df.loc[i, "acc_rtn"] = acc_rtn
print("누적 수익율 : ", acc_rtn)
    


### 절대 모멘텀 모듈 만들기
1. ```__init__``` 함수에서 데이터프레임과 기준이 되는 컬럼의 이름을 받아온다. 
2. 파생변수 'STD-YM' 컬럼을 추가하는 함수 생성
3. 새로운 데이터프레임 month_last_df을 생성하고 'BF_1M', 'BF_12M' 파생변수를 생성하는 함수를 생성
4. 거래 내역을 추가하는 함수를 생성
5. 수익율을 계산하는 함수를 생성